In [21]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import *
from datetime import datetime
import json
import collections
import logging
import os
import sys

#### Handling logs

In [3]:
today_date = pd.to_datetime('today').strftime("%Y_%m_%d-%H_%M_%S")
log_folder = 'logs'
if not os.path.exists(log_folder):
            os.makedirs(log_folder)
logging.basicConfig(filename=f'{log_folder}/{today_date}.log', level=logging.INFO)
logger = logging.getLogger(__name__)

#### Class to handle tasks related to reading and writing JSON files

In [4]:
class json_basic:
    def __init__(self) -> None:
        pass
        
    def readJson(self, file_name:str, folder_name:str = "")->dict:
        '''Returns a dictionary of the information in JSON file. If file doesn't exist, then return a None value.'''
        if(type(file_name) == pd.Timestamp):
            file_name = file_name.date()
        
        if(folder_name == ""):
            try:
                with open(f"{file_name}.json", 'r') as file:
                    data = json.load(file)
                logger.info(f"{file_name}.json from {folder_name} has been read")
                return data
            except:
                logger.error(f"{file_name}.json not avaliable in the {folder_name}")
                return None
            
        try:
            with open(f"{folder_name}/{file_name}.json", 'r') as file:
                data = json.load(file)
            logger.info(f"{file_name}.json has been read")
            return data
        except:
            logger.error(f"{file_name}.json not avaliable at the location")
            return None
        
    def writeJson(self, dt:dict, file_name:str, folder_name:str = "")->None:
        '''Writes the dictionary in the JSON file, with given file name and folder name.'''
        if(type(file_name) == pd.Timestamp):
            file_name = file_name.date()

        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        json_object = json.dumps(dt, indent=4)
        if(folder_name == ""):
            with open(f"{file_name}.json", "w") as outfile:
                outfile.write(json_object)
                logger.info(f"{file_name}.json has been added to the same folder")
        else:
            with open(f"{folder_name}/{file_name}.json", "w") as outfile:
                outfile.write(json_object)
                logger.info(f"file {file_name}.json is added to {folder_name}")

    def check(self, dt) -> bool:
        '''Returns False if dt contains NULL value, returns True otherwise'''
        if(np.isscalar(dt)):
            if(pd.isna(dt)):
                return False
        if(isinstance(dt, dict)):
            for ele in dt.values():
                if self.check(ele) == False:
                    return False
        if(isinstance(dt, list)):
            for ele in dt:
                if self.check(ele) == False:
                    return False
        return True

jb = json_basic()

In [5]:
data = jb.readJson("parameters")
#converting dates from str datatype to datetime datatype
for key in data:
    if key[-4:] == "date":
        data[key] = pd.to_datetime(data[key])
    if key[-5:] == "dates":
        for i, ele in enumerate(data[key]):
            data[key][i] = pd.to_datetime(ele)

ppn = data['price_prcsn']
ipn = data['index_prcsn']
calculation_days = pd.date_range(data['st_date'], data['end_date'])

In [6]:
df = pd.read_excel(data['underlying_file_path'], sheet_name=None, index_col=0)
all_dates = df['GLD'].index.to_list()

#### Class to create json files from dataframe

In [7]:
class create_json(json_basic):
    def __init__(self, df:pd.DataFrame):
        '''Parameters:
            df: Dataframe containing information about assets'''
        self.__df = df

    def create(self, date: pd.Timestamp)->dict:
        '''Creating JSON files of given date using data given in uderlying dataframe\n
        Parameters:
            date: date of calculation day'''
        df = self.__df
        dt = collections.defaultdict(dict)
        if(date > data['ref_app_date']):
            dt['REF']['RATE'] = df['SOFR'].loc[date, 'SOFR']
        else:
            dt['REF']['RATE'] = df['FFER'].loc[date, 'DFF']
        for ast in data['assets']:
            dt[ast]['Close'] = df[ast].loc[date, 'Close']
            val = 0
            div_name = str(ast)+"_Divident"
            if div_name in df.keys() and date in df[div_name].index:
                val = df[div_name].loc[date, 'Dividends']
            dt[ast]['Dividend'] = val
        
        if(self.check(dt) == False):
            logger.error(f"Complete underlying data for date: {date} is not present")
            sys.exit(0)
        
        self.writeJson(dt, date, data['folder_name'])
        return dt

cj = create_json(df.copy())

#### Class to update json files with required features

In [8]:
class asset_chr(json_basic):
    def __init__(self, cj:create_json):
        self.cj = cj

    def next_date(self, date: pd.Timestamp, n:int):
        '''Returns a date "n" buisness days after the given date\n
        Parameters:
            date: beginning date for calculation
            n: number of buisness days'''
        if date in all_dates:
            i = all_dates.index(date)
            if(i+n < len(all_dates) and i+n >= 0):
                return all_dates[i+n]
        return date + BusinessDay(n)

    def readJsonCustom(self, date:pd.Timestamp, folder_name:str, completeIt: bool = False) -> dict:
        '''Reads and returns json file in dictionary format, creates new file if file doesn't exist\n
        Parameters:
            date: Name of the json file
            folder_name: destination of the file
            completeIt: True if operations of given class are to be executed'''
        res = self.readJson(date, folder_name)
        if res == None:
            res = self.cj.create(date)
        if completeIt and 'RANK' not in res.keys():
            res = self.calculate(date)
        return res
    
    def calcAF(self, ast:str, date: pd.Timestamp, todays:dict)->float:
        '''Returns the adjustment factor of given asset and given date\n
        Parameters:
            ast: name of asset
            date: date of calculation day
            todays: dictionary containing today's information of index's assets'''
        res = (todays[ast]['Close'] + todays[ast]['Dividend']*(1-data['dividend_tax']))/todays[ast]['Close']
        logger.info(f'Adjustment factor for asset: {ast} on date: {date} is {res}')
        return res
        
    def calcAAL(self, ast:str, date: pd.Timestamp, todays:dict)->float:
        '''Returns the asset adjustment level of given asset and given date\n
        Parameters:
            ast: name of asset
            date: date of calculation day
            todays: dictionary containing today's information of index's assets '''
        if(date < data['aal_start_date']):
            return None

        if(date == data['aal_start_date']):
            return 100
        
        prev_date = self.next_date(date, -1)
        prev_date_data = self.readJsonCustom(prev_date, data['folder_name'], True)            
        res = prev_date_data[ast]['AAL']*todays[ast]['AF']*todays[ast]['Close']/prev_date_data[ast]['Close']
        logger.info(f'Asset adjusted level for asset: {ast} on date: {date} is {res}')
        return res

    def calcAALForCash(self, date: pd.Timestamp)->float:
        '''Returns the AAL for referrence rate to be used as cash index\n
        Parameters:
            date: date of calculation day'''
        if(date < data['aal_start_date']):
            return None

        if(date == data['aal_start_date']):
            return 1000        
        
        prev_date = self.next_date(date, -1)
        prev_date_data = self.readJsonCustom(prev_date, data['folder_name'], True)
        
        time_in_yr = (date-prev_date).days/360
        res = prev_date_data['REF']['AAL']*(1 + prev_date_data['REF']['RATE']*time_in_yr/100)
        logger.info(f'Cash Index for date: {date} is {res}')
        return res
    
    def calcMS(self, ast:str, date: pd.Timestamp, todays:dict)->float:
        '''Returns momentum score of given assest on a given date\n
        Parameters:
            ast: name of asset
            date: date of calculation day
            todays: dictionary containing today's information of index's assets '''
        mnt_back_date = date - pd.DateOffset(months=1)
        yr_back_date = date - pd.DateOffset(months=12)

        if(yr_back_date < all_dates[0]):
            return None

        while yr_back_date not in all_dates:
            yr_back_date = yr_back_date - BusinessDay(1)

        while mnt_back_date not in all_dates:
            mnt_back_date = mnt_back_date - BusinessDay(1)

        yr_back_date_data = self.readJsonCustom(yr_back_date, data['folder_name'], True)
        mnt_back_date_data = self.readJsonCustom(mnt_back_date, data['folder_name'], True)

        if(yr_back_date_data[ast]['AAL'] == None):
            return None

        res = mnt_back_date_data[ast]['AAL']/yr_back_date_data[ast]['AAL']
        logger.info(f'Momentum Scores for date: {date} is {res}')
        return res

    def calcRank(self, todays:dict)->dict:
        '''Returns rank of every assest in descending order\n
        Parameters:
            todays: dictionary containing today's information of index's assets '''
        row = []
        any_none_val = False
        for ast in data['assets'] + ['REF']:
            val = todays[ast]['MS']
            row.append(val)
            if(val == None):
                any_none_val = True
        
        if(any_none_val):
            return dict(zip(data['assets']+['REF'], row))
        
        srtd = row.copy()
        srtd.sort(reverse=True)
        row2 = [None]*len(row)
        for i, num in enumerate(srtd):
            for j, ms in enumerate(row):
                if(ms == num):
                    row2[j] = i+1
                    break

        res = dict(zip(data['assets']+['REF'], row2))
        logger.info(f'Ranks are {res}')
        return res
    
    def calculate(self, date: pd.Timestamp)->dict:
        '''This function calculates and returns adjustment factor, assest adjustment level, momentum score and rank for each asset. It requires date of the given day, and basic data of the day in dictionary format\n
        Parameters:
            date: date of calculation day
            todays: dictionary containing today's information of index's assets '''
        logger.info(f"Started creating features from the assets for date: {date}")
        dict = self.readJsonCustom(date, data['folder_name'])
        
        for ast in data['assets']:
            dict[ast]["AF"] = self.calcAF(ast, date, dict)
            dict[ast]["AAL"] = self.calcAAL(ast, date, dict)
            dict[ast]["MS"] = self.calcMS(ast, date, dict)    
        dict["REF"]["AAL"] = self.calcAALForCash(date)
        dict["REF"]["MS"] = self.calcMS("REF", date, dict)
        dict['RANK'] = self.calcRank(dict)
        
        self.writeJson(dict, date, data['folder_name'])
        logger.info(f"Finished creating features from the assets for date: {date}")
        return dict

#### Class to calculate sub-index

In [9]:
class subIndices(json_basic):
    def __init__(self, inception_date:pd.Timestamp, assets:list, name:str, ac:asset_chr):
        '''Parameters:
            inception_date: Inception Date of the index
            Assets: List of assets of respective index
            name: Name of the index
            ac: Object of class which creates features of assets'''
        self.in_date = inception_date
        self.assets = assets   
        self.name = name
        self.ac = ac

    def price(self, dt:dict, ast:str)->float:
        '''Returns the closing price of given assest in given dictionary\n
        Parameters:
            dt: dictionary containing today's portfolio of respective index only
            ast: name of the asset'''
        if not dt:
            return 0
        return round(dt[ast]['Close'],ppn)
    
    def next_date(self, date: pd.Timestamp, n:int):
        '''Returns a date "n" buisness days after the given date\n
        Parameters:
            date: beginning date for calculation
            n: number of buisness days'''
        if date in all_dates:
            i = all_dates.index(date)
            if(i+n < len(all_dates) and i+n >= 0):
                return all_dates[i+n]
        return date + BusinessDay(n)
    
    def calc_rday(self, date: pd.Timestamp)->bool:
        '''Returns whether a calculation day is a rebalance day or not\n
        Parameters:
            date: date of calculation day'''
        if(date < self.in_date):
            return None
            
        nextDate = self.next_date(date,1)
        rday = False
        if(date.month == self.in_date.month and date.month != nextDate.month):
            rday = True
        logger.info(f"Date:{date} is marked {rday} for rebalance day")
        return rday
    
    def calc_sday(self, date: pd.Timestamp)->bool:
        '''Returns whether a given day is a selection day or not\n
        Parameters:
            date: date of calculation day'''
        Date2 = self.next_date(date,2)
        if(Date2 < self.in_date):
            return None
        Date3 = self.next_date(Date2, 1)
        sday = False
        if(Date2.month == self.in_date.month and Date2.month != Date3.month):
            sday = True
        logger.info(f"Date:{date} is marked {sday} for selection day")
        return sday

    def calc_u(self, ast:str, date: pd.Timestamp, todays:dict, pdd:dict, dt:dict)->float:
        '''Calculates and returns the asset unit weight of given assest\n
        Parameters:
            ast: Name of asset
            date: date of calculation day
            todays: dictionary containing today's information of index's assets
            dt: dictionary containing today's portfolio of respective index only
            prev_data_dict: dictionary containing portfolio of last calculation day'''
        u = 0
        if(date < self.in_date):
            return u
        
        if(date == self.in_date):
            u = dt["AW"][ast]*1000/todays[ast]['Close']
        elif(dt["R_DAY"]):
            u = dt["AW"][ast]*pdd[self.name]['SIL']/self.price(pdd, ast)
        else:
            u = todays[ast]['AF']*pdd[self.name]['U'][ast]
        logger.info(f"Units of assest: {ast} for date:{date} is {u}")
        return u

    def calc_aw(self, ast:str, date: pd.Timestamp, todays:dict, pdd:dict, dt:dict)->float:
        '''Calculates and returns the asset weight of given assest\n
        Parameters:
            ast: Name of asset
            date: date of calculation day
            todays: dictionary containing today's information of index's assets
            dt: dictionary containing today's portfolio of respective index only
            prev_data_dict: dictionary containing portfolio of last calculation day'''
        if(dt['S_DAY'] == False):
            return pdd[self.name]['AW'][ast]
        
        ast_rnk = todays['RANK'][ast]
        aw = None
        if(ast_rnk == None or ast_rnk > todays["RANK"]["REF"]):
            aw = 0
        elif(ast_rnk == 1):
            aw = 0.6
        elif(ast_rnk == 2):
            aw = 0.4
        else:
            aw = 0
        logger.info(f"RANK of assest: {ast} for date: {date} is {aw}")
        return aw

    def calc_cost(self, date: pd.Timestamp, todays:dict, prev_date_data:dict, dt:dict)->float:
        '''Calculates and returns the cost of given calculation day\n
        Parameters:
            date: date of calculation day
            todays: dictionary containing today's information of index's assets
            dt: dictionary containing today's portfolio of respective index only
            prev_data_dict: dictionary containing portfolio of last calculation day'''
        cost = 0
        if(date <= self.in_date):
            return 0
        pdd = prev_date_data
        prev_date = self.next_date(date, -1)
        for ast in self.assets:
            if(dt['R_DAY']):
                cost += round(todays[ast]['Close'],ppn) * abs(dt['U'][ast] - pdd[self.name]['U'][ast])*data['TCR']
            cost += self.price(pdd, ast)*pdd[self.name]['U'][ast]*(date-prev_date).days*(data['spread'] + pdd['REF']['RATE']/100)/360
        logger.info(f"COST for date: {date} is {cost}")
        return cost
    
    def calc_sil(self, date: pd.Timestamp, todays:dict, prev_date_data:dict, dt:dict)->float:
        '''Calculates and returns the sub-index level of given calculation day\n
        Parameters:
            date: date of calculation day
            todays: dictionary containing today's information of index's assets
            dt: dictionary containing today's portfolio of respective index only
            prev_data_dict: dictionary containing portfolio of last calculation day'''
        sil = 0
        if(date < self.in_date):
            return None
        if(date == self.in_date):
            return 1000
        
        pdd = prev_date_data
        sil = pdd[self.name]['SIL'] - dt['COST']                 
        for ast in self.assets:
            sil += pdd[self.name]['U'][ast]*(todays[ast]['AF']*round(todays[ast]['Close'],ppn) - self.price(pdd, ast))
        logger.info(f"SIL for date: {date} is {sil}")
        return sil
    
    def completeIt(self, date:pd.Timestamp, il: bool)-> dict:
        '''Reads and returns json file in dictionary format, creates new file if file doesn't exist\n
        Parameters:
            date: Name of the json file
            completeIt: True if index value of given class is required'''
        dt = self.readJson(date, data['folder_name'])
        for ast in self.assets+['RANK']:
            if dt == None or (ast not in dt.keys()):
                dt = self.ac.calculate(date)

        if(il and self.name not in dt.keys()):
            dt = self.calculate(date)
        return dt

    def calculate(self, date: pd.Timestamp)->dict:
        '''It takes date of the calculation day and asset's data as "todays", and return a updated dictionary with sub-index portfolio calculation on the given date\n
        Parameters:
            date: date of calculation day
            todays: dictionary containing today's information of index's assets '''
        logger.info(f"Started creating {self.name} from the assets for date: {date}")

        todays = self.completeIt(date, False)
        
        dt = collections.defaultdict(dict)
        dt['S_DAY'] = self.calc_sday(date)
        if(dt['S_DAY'] == None):
            logger.warning(f"{self.name} doesn't exist on date:{date} ")
            return todays

        prev_date = self.next_date(date, -1)
        prev_date_data = self.completeIt(prev_date, True)

        dt['R_DAY'] = self.calc_rday(date)
        for ast in self.assets:
            dt["AW"][ast] = self.calc_aw(ast, date, todays, prev_date_data, dt) 
            dt["U"][ast] = self.calc_u(ast, date, todays, prev_date_data, dt)
        dt["COST"] = self.calc_cost(date, todays, prev_date_data, dt)
        dt["SIL"] = self.calc_sil(date, todays, prev_date_data, dt)

        todays[self.name] = dt
        self.writeJson(todays, date, data['folder_name'])
        logger.info(f"Finished creating {self.name} from the assets for date: {date}")
        return todays

#### Class for index calculation

In [10]:
class RBCIndex(json_basic):
    def __init__(self, inception_date: pd.Timestamp, name:str, si:list[subIndices]):
        '''Parameters:
            inception_date: Inception Date of the index
            name: Name of the index
            si: List of objects of the subinex class'''
        self.in_date = inception_date
        self.name = name
        self.si = si

    def sil(self, dt:dict, num:int) -> float:
        '''Returns sil of given number in given dictionary \n
        Parameters:
            dt: dictionary containing today's portfolio of respective index only
            num: number associated with underlying subindex'''
        if not dt:
            return 0
        if self.name not in dt.keys():
            return 0
        return dt[self.name][f'SIL_{num}']
    
    def siu(self, dt:dict, num:int) -> float:
        '''Returns siu of given number in given dictionary\n
        Parameters:
            dt: dictionary containing today's portfolio of respective index only
            num: number associated with underlying subindex'''
        if not dt:
            return 0
        if self.name not in dt.keys():
            return 0
        return dt[self.name][f'SIU_{num}']
    
    def next_date(self, date: pd.Timestamp, n:int) -> pd.Timestamp:
        '''Returns a date "n" buisness days after the given date\n
        Parameters:
            date: beginning date for calculation
            n: number of buisness days'''
        if date in all_dates:
            i = all_dates.index(date)
            if(i+n < len(all_dates) and i+n >= 0):
                return all_dates[i+n]
        return date + BusinessDay(n)
    
    def calc_rday(self, date: pd.Timestamp) -> bool:
        '''Calculates and returns whether a date is a rebalance day or not\n
        Parameters:
            date: date of calculation day'''
        if(date < self.in_date):
            return None
            
        nextDate = self.next_date(date,1)
        rday = False
        if(date.month == self.in_date.month and date.month != nextDate.month):
            rday = True
        logger.info(f"Date: {date} is marked as {rday} as rebalance day for {self.name}")
        return rday

    def calc_siu(self, num:int, date: pd.Timestamp, dt:dict, prev_date_data:dict) -> float:
        '''Calculates and returns the siu for "date"'s portfolio \n
        Parameters:
            num: number associated with underlying subindex
            date: date of calculation day
            dt: dictionary containing today's portfolio of respective index only
            prev_data_dict: dictionary containing portfolio of last calculation day'''
        siu = None
        pdd = prev_date_data
        if(date == self.in_date):
            siu = 1000/(12*dt[f'SIL_{num}'])
        elif(dt['R_DAY']):
            siu = pdd[self.name]['IL']/(12*self.sil(pdd, num))
        else:
            siu = self.siu(pdd, num)    
        logger.info(f"SIU for SI_{num} is {siu} on date: {date} for {self.name}")
        return siu

    def calc_il(self, date: pd.Timestamp, dt:dict, prev_date_data:dict) -> float:
        '''Calculates and returns the index level for "date"'s portfolio \n
        Parameters:
            date: date of calculation day
            dt: dictionary containing today's portfolio of respective index only
            prev_data_dict: dictionary containing portfolio of last calculation day'''
        il = None
        if(date == self.in_date):
            il = 1000
        else:
            pdd = prev_date_data   
            il = pdd[self.name]['IL']
            for num in range(1,13):
                il += self.siu(pdd, num)*(dt[f'SIL_{num}'] - self.sil(pdd, num))
            il = round(il,ipn)

        logger.info(f"Index level is {il} on date: {date} for {self.name}")
        return il
    
    def completeIt(self, date:pd.Timestamp, il:bool) -> dict:
        '''Reads and returns json file in dictionary format, creates new file if file doesn't exist\n
        Parameters:
            date: Name of the json file
            il: True if index level of given class is required'''
        dt = self.readJson(date, data['folder_name'])
        for i in range(1, 13):
            asset_name = f"SI_{i}"
            if dt==None or asset_name not in dt.keys():
                logger.warning(f"Data of {asset_name} not complete for RBC Index on {date} ")
                dt = self.si[i].calculate(date)

        if(il and self.name not in dt.keys()):
            logger.warning(f"Data not complete for RBC Index on {date} ")
            dt = self.calculate(date)
        return dt
     
    def calculate(self, date: pd.Timestamp) -> dict:
        '''Calculates index level and other required parameter and returns an updated dictionary with date's portfolio\n
        Parameters:
            date: date of calculation day
            todays: dictionary containing today's information of index's assets '''
        logger.info(f'Started creating {self.name} for date:{date}')
        
        todays = self.completeIt(date, False)
        dt = {}
        dt['R_DAY'] = self.calc_rday(date)
        if(dt['R_DAY'] == None):
            logger.warning(f"{self.name} index doesn't exist for date: {date}")
            return todays
      
        prev_date = self.next_date(date, -1)
        prev_date_data = self.completeIt(prev_date, True)

        for num in range(1,13):
            asset_name = f"SI_{num}"
            dt[f'SIL_{num}'] = todays[asset_name]['SIL']
            dt[f'SIU_{num}'] = self.calc_siu(num, date, dt, prev_date_data)
        dt["IL"] = self.calc_il(date, dt, prev_date_data)

        todays[self.name] = dt
        self.writeJson(todays, date, data['folder_name'])
        logger.info(f'Finished creating {self.name} for date:{date}')
        return todays

In [11]:
cj_temp = create_json(df)

ac_temp = asset_chr(cj_temp)

si = [None]*13
for i, inc_date in enumerate(data['si_inc_dates'],1):
    si[i] = subIndices(inc_date, data['assets'], f"SI_{i}", ac_temp)

il1 = RBCIndex(data['ind_inc_date'], 'RBC', si)

<style>
div.warn {    
    background-color: #dfb5b4;
    border: 5px solid #dfb5b4;
    padding: 0.5em;
    }
 </style>
<div class="warn">
    Warning: Below code will change the JSON files
</div>

#### Creating JSON files with asset data, creating sub-indices, and indices

In [12]:
# for date in calculation_days:
#     if date not in all_dates:
#         if(date > all_dates[-1] or date < all_dates[0]):
#             logger.warning(f"Data not available for {date}")
#         else:
#             logger.warning(f"{date} is not a buisness day")
#         continue
#     il1.calculate(date) 

In [24]:
def test_func(start:pd.Timestamp, end:pd.Timestamp) -> pd.DataFrame:
    # if( isinstance(start, str)):
    if( not(isinstance(start, (pd.Timestamp, np.datetime64, datetime)))):
        raise ValueError('start_date argument is not a datetime type object')
    if( not(isinstance(end, (pd.Timestamp, np.datetime64, datetime)))):
        raise ValueError('end_date argument is not a datetime type object')
    
    days_range = pd.date_range(start, end)
    res = pd.DataFrame(columns=['Date', 'Index'])
    for day in days_range:
        if day < data['ind_inc_date']:
            il = "Index do not exist"
        elif day not in all_dates:
            if day > all_dates[-1]:
                il = "Out of Bound"
            else:
                il = "Holiday"
        else:
            # il = jb.readJson(day, "indexes")['RBC']['IL']
            il = il1.calculate(day)['RBC']['IL'] 
        res.loc[res.shape[0]] = [day, il]
    res.set_index('Date', inplace=True)
    return res

In [27]:
# st = pd.to_datetime('2023-12-02')
# end = pd.to_datetime('2023-12-09')
st = datetime(2023,12,5)
end = datetime(2023,12,9)
z = testfunc(st, end)

In [28]:
z

,Index
Date,
2023-12-05,2424.388
2023-12-06,2424.22
2023-12-07,2430.572
2023-12-08,2422.863
2023-12-09,Out of Bound
